In [599]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [600]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"UNFI"}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':126.365}, 
             {'symbol':"EZPW", 'basis':15.46}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':66.113}, 
             {'symbol':"ITRN"}, 
             {'symbol':"CRDO", 'basis':35.613}, 
             {'symbol':"PYPL"}, 
             {'symbol':"ALL"}, 
             {'symbol':"LC"}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':69.342}, 
             {'symbol':"CCL", 'basis':17.999}, 
             {'symbol':"AGX", 'basis':93.089}, 
             {'symbol':"POWL", 'basis':187.000}, 
             {'symbol':"PPC", 'basis':45.45}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':83.177}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA"}, 
             {'symbol':"TWLO", 'basis':100.519}, 
             {'symbol':"APP", 'basis':276.123}, 
             {'symbol':"TMUS"}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':54.777}, 
             {'symbol':"CAAP", 'basis':21.57}
             ]

# Brokerage
brokerage_funds = [{'symbol':"AGX", 'basis':86.654}, 
                   {'symbol':"APP", 'basis':225.619}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"CCL", 'basis':20.829}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"CRDO", 'basis':38.629}, 
                   {'symbol':"EAT", 'basis':135.74}, 
                   {'symbol':"EZPW", 'basis':18.187}, 
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.194}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"WFC", 'basis':72.705}
                   ]

In [601]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.9):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [602]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=1, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [603]:
brokerage_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,AGX,86.654,203.26,136.91,69.93,146.55,33.00,7.880,0.0074,Sell
1,APP,225.619,354.29,278.74,17.01,307.40,64.07,208.350,0.0000,Sell
2,ARQT,15.996,13.98,13.88,13.88,15.18,inf,11.680,0.0000,Hold
3,ATGE,86.385,128.50,105.81,85.18,111.10,21.74,3.222,0.0000,Sell
4,CCL,20.829,22.25,19.39,17.98,20.43,14.35,3.283,0.0000,Sell
5,CLS,74.924,119.47,86.48,38.63,93.45,33.37,8.820,0.0000,Sell
6,CRDO,38.629,61.78,44.23,1.83,48.43,2059.33,16.970,0.0000,Sell
7,EAT,135.740,154.53,140.13,35.91,148.97,21.49,26.520,0.0000,Sell
8,EZPW,18.187,13.40,14.88,22.59,15.46,11.26,0.962,0.0000,Hold
9,FBTC,75.244,94.98,76.00,NaN,79.01,NaN,NaN,NaN,Sell


In [ ]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=0.975, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [605]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,EAT,126.365,154.53,140.13,35.91,148.97,21.49,26.520,0.0000,Sell
3,EZPW,15.460,13.40,14.88,22.59,15.46,11.26,0.962,0.0000,Hold
4,ARQT,15.932,13.98,13.88,13.88,15.18,inf,11.680,0.0000,Hold
5,WFC,66.113,72.83,68.47,92.42,71.02,13.10,1.459,0.0220,Sell
7,CRDO,35.613,61.78,44.23,1.83,48.43,2059.33,16.970,0.0000,Sell
12,CLS,69.342,119.47,86.48,38.63,93.45,33.37,8.820,0.0000,Sell
13,CCL,17.999,22.25,19.39,17.98,20.43,14.35,3.283,0.0000,Sell
14,AGX,93.089,203.26,136.91,69.93,146.55,33.00,7.880,0.0074,Sell
15,POWL,187.000,174.70,168.14,140.22,179.38,12.27,3.894,0.0061,Hold
16,PPC,45.450,49.60,49.85,45.46,51.91,9.76,3.754,0.0000,Hold


In [606]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,UNFI,0.0,29.34,26.28,26.28,27.70,inf,1.093,0.0000,Sell
1,MFC,0.0,31.21,29.56,33.33,30.54,16.43,1.643,0.0407,Sell
6,ITRN,0.0,37.26,36.75,27.82,38.38,13.80,4.002,0.0537,Hold
8,PYPL,0.0,69.85,65.65,53.40,68.09,15.70,3.354,0.0000,Sell
9,ALL,0.0,203.24,196.24,184.67,201.97,13.89,2.684,0.0197,Sell
10,LC,0.0,9.86,10.22,12.72,10.81,22.41,0.825,0.0000,Hold
11,QTWO,0.0,86.57,78.01,78.01,81.47,inf,9.890,0.0000,Sell
17,SYF,0.0,56.74,51.23,95.24,53.78,7.77,1.406,0.0211,Sell
18,ATGE,0.0,128.50,105.81,85.18,111.10,21.74,3.222,0.0000,Sell
19,BRK-B,0.0,503.46,507.00,591.79,520.70,13.42,1.660,0.0000,Hold


In [607]:
# Research new investments
watch_list = [{'symbol':'COMM'}, # Value play?
              {'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'} # Mexico - Communications - mobile network
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [608]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,COMM,0,5.70,4.69,4.69,5.13,inf,30.360,0.0000,Sell
1,BSAC,0,25.04,22.96,26.30,23.56,11.03,2.529,0.0539,Sell
2,KARO,0,52.86,42.28,17.16,44.85,31.84,9.170,0.0204,Sell
3,QFIN,0,42.52,40.64,68.87,43.06,6.57,1.786,0.0306,Hold
4,TSM,0,191.98,165.43,86.52,172.47,23.10,6.560,0.0171,Sell
5,CCU,0,13.86,14.42,16.05,14.90,14.59,1.574,0.0329,Hold
6,ABEV,0,2.52,2.32,2.30,2.38,15.75,2.353,0.0606,Sell
7,TIMB,0,16.97,15.24,18.63,15.60,13.47,1.895,0.0910,Sell
8,ASR,0,345.93,289.26,258.78,299.78,14.83,3.709,0.0711,Sell
9,RYAAY,0,56.03,46.83,38.98,48.15,17.03,3.734,0.0167,Sell


In [609]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [610]:
mag7_test_table

[['MSFT', 0, 450.18, 352.71, 131.27, 402.86, 34.84, 10.39, 0.0074, 'Sell'],
 ['AAPL', 0, 195.27, 183.28, 29.73, 212.21, 30.42, 43.66, 0.0053, 'Hold'],
 ['NVDA', 0, 131.29, 99.41, 17.16, 116.78, 44.66, 40.36, 0.0003, 'Sell'],
 ['AMZN', 0, 200.99, 168.88, 73.71, 195.47, 32.79, 6.98, 0, 'Sell'],
 ['GOOG', 0, 169.59, 143.78, 88.53, 165.78, 18.95, 5.96, 0.005, 'Sell'],
 ['GOOGL', 0, 168.47, 142.13, 88.6, 163.92, 18.8, 5.92, 0.005, 'Sell'],
 ['META', 0, 627.06, 512.53, 240.74, 594.68, 24.51, 8.52, 0.0033, 'Sell'],
 ['BRK-B', 0, 503.46, 456.3, 591.79, 520.7, 13.42, 1.66, 0, 'Hold'],
 ['TSLA', 0, 339.34, 236.01, 35.44, 281.0, 192.81, 14.64, 0, 'Sell']]

In [611]:
# Machine Learning

In [612]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [613]:
# Feature selection and normalization

In [614]:
# Build model

In [615]:
# Train model

In [616]:
# Evaluate model

In [617]:
# Refine model through hyperparameter tuning